# Cryptocurrency Prophet

In [ ]:
!pip install pandas prophet requests ipywidgets

In [12]:
#@title Using Python Prophet and CoinCap for crpytocurrency predictions.
#@markdown Created a list of all the available currencies via CoinCap.<br>
#@markdown Used historical pricing on a 1 day basis as our dataset.

import logging
logging.basicConfig()
logger = logging.getLogger('foo')
logger.setLevel(logging.ERROR)

import sys
from datetime import datetime
from typing import List, Set

import pandas as pd
import requests
from prophet import Prophet


proxies = {
    # 'https': 'https://116.254.116.99:8080'
}

requests.Session().get('https://api.coincap.io/v2/assets', proxies=proxies)

def get_supported_currencies_datasets() -> List[str]:
    try:
      datasets = requests.Session().get('https://api.coincap.io/v2/assets', proxies=proxies).json()
    except Exception as e:
      print(e)
      print('Failed to get the list of currencies from coincap. Please retry.')
      return
    coins = [f['id'] for f in datasets['data']]
    return coins


def predict_currencies(currencies: Set[str]):
    currencies = [c for c in currencies if c]
    futures = []
    forecasts = []
    for currency_name in currencies:
        print("Forcasting", currency_name)
        # Get the dataset
        try:
          historical_data = requests.Session().get(f'https://api.coincap.io/v2/assets/{currency_name}/history?interval=d1', proxies=proxies).json()[
              'data']
        except Exception:
          print(f'Failed to get the data for {currency_name} from coincap. Please retry.')
          return

        df = pd.DataFrame.from_dict(historical_data)

        # Normalization
        print("Normalizing the data...")
        df.rename(columns={"time": "ds", "priceUsd": "y"}, inplace=True)
        # Stripping timezone
        df['ds'] = df['ds'].apply(lambda timestamp: datetime.fromtimestamp(int(timestamp) / 1000))
        df['ds'] = df['ds'].apply(lambda d: d.replace(tzinfo=None))
        df['ds'] = pd.to_datetime(df['ds'])
        df['ds'].dt.tz_localize(None)

        # Fit
        print("Creating a time-series model...")
        m = Prophet(seasonality_mode='multiplicative', daily_seasonality=True, yearly_seasonality=True).fit(df)

        # Predictions
        future = m.make_future_dataframe(periods=90, include_history=False)
        forecast = m.predict(future)
        futures.append(future)
        forecasts.append(forecast)
        print(f"Forecast for {currency_name}: ")
        print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())
        forecast_plt = m.plot(forecast)
        forecast_gca = forecast_plt.gca()
        forecast_gca.set_title(f"{currency_name.upper()} Forecast", size=34)
        forecast_plt.show()
        # components_plt = m.plot_components(forecast)
        # components_gca = components_plt.gca()
        # components_gca.set_title(f"{currency_name.upper()} Components", size=34)
        # components_plt.show()

    return forecasts

In [18]:
#@title List all currencies fetched on CoinCap
#@markdown Select a crypto
supported_currencies = get_supported_currencies_datasets()
if supported_currencies:
  import ipywidgets as widgets
  selection = widgets.SelectMultiple(
      options=supported_currencies,
      value=['ethereum'],
      rows=100,
      description='Currencies',
      disabled=False
  )
  display(selection)

SelectMultiple(description='Currencies', index=(1,), options=('bitcoin', 'ethereum', 'tether', 'binance-coin',…

In [ ]:
#@title Run the predictions on the selected currencies
currencies = selection.value
print('Predicting ',currencies)
forecasts = predict_currencies(selection.value)